In [1]:
       """ for i in new_df['ID'].unique():
            temp_df=new_df[new_df['ID']==i].reset_index(drop=True)
            temp_df["height"].fillna(method='bfill',inplace=True)
            temp_df["weight"].fillna(method='bfill',inplace=True)
            temp_df["height"].fillna(method='ffill',inplace=True)
            temp_df["weight"].fillna(method='ffill',inplace=True)
            temp2.append(temp_df)
        new_df=pd.concat(temp2,axis=0)  """

' for i in new_df[\'ID\'].unique():\n     temp_df=new_df[new_df[\'ID\']==i].reset_index(drop=True)\n     temp_df["height"].fillna(method=\'bfill\',inplace=True)\n     temp_df["weight"].fillna(method=\'bfill\',inplace=True)\n     temp_df["height"].fillna(method=\'ffill\',inplace=True)\n     temp_df["weight"].fillna(method=\'ffill\',inplace=True)\n     temp2.append(temp_df)\n new_df=pd.concat(temp2,axis=0)  '

In [2]:
import pandas as pd
import torch
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np
%matplotlib inline

In [3]:
"""# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)"""

"# row 생략 없이 출력\npd.set_option('display.max_rows', None)\n# col 생략 없이 출력\npd.set_option('display.max_columns', None)"

In [4]:
df=pd.read_csv("../../../Integrated_data/merged_df_reward.csv",encoding="cp949")

In [5]:
import pandas as pd

class DataProcessor:
    def __init__(self, df):
        self.df = df
        self.abmormaled()
        #self.interpolated()
        self.result=self.fill_missing_values()

    # ... (다른 클래스 함수 정의는 여기에 있어야 합니다)

    def interpolated(self):
        temp=[]
        temp2=[]
        for i in df['ID'].unique():
            try:
                temp.append(df[df['ID']==i].interpolate())
            except:
                temp.append(df[df['ID']==i])

        new_df=pd.concat(temp,axis=0)
        new_df.fillna(method='bfill',inplace=True)   
        for i in new_df['ID'].unique():
            temp_df=new_df[new_df['ID']==i].reset_index(drop=True)
            temp_df["height"].fillna(method='bfill',inplace=True)
            temp_df["weight"].fillna(method='bfill',inplace=True)
            temp_df["height"].fillna(method='ffill',inplace=True)
            temp_df["weight"].fillna(method='ffill',inplace=True)
            temp2.append(temp_df)
        new_df=pd.concat(temp2,axis=0)  
        self.df = new_df   
    def gap(self,first, last):
        gap_per=(last-first)/first*100
        return abs(gap_per)
        
    def abmormaled(self):
        temp = []
        for i in self.df['ID'].unique():
            temp_df = self.df[self.df['ID'] == i].reset_index(drop=True)
            temp_df["height"].fillna(method='bfill',inplace=True)
            temp_df["weight"].fillna(method='bfill',inplace=True)
            temp_df["height"].fillna(method='ffill',inplace=True)
            temp_df["weight"].fillna(method='ffill',inplace=True)
            for i in range(1, len(temp_df)):
                if temp_df.at[i, 'height'] <= temp_df.at[i - 1, 'height']:
                    temp_df.at[i, 'height'] = temp_df.at[i - 1, 'height']
                if self.gap(temp_df.at[i - 1, 'height'], temp_df.at[i, 'height']) >= 16:       
                    temp_df.at[i, 'height'] = temp_df.at[i - 1, 'height']
                if self.gap(temp_df.at[i - 1, 'height'], temp_df.at[i, 'height']) >= 16:       
                    temp_df.at[i, 'height'] = temp_df.at[i - 1, 'height']
            temp.append(temp_df)
        new_df2 = pd.concat(temp, axis=0)
        new_df2.reset_index(drop=True, inplace=True)
        self.df = new_df2

    def fill_missing_values(self):
        self.df["step_count"].fillna(self.df["step_count"].median(), inplace=True)
        self.df["burned calory"].fillna(self.df["burned calory"].median(), inplace=True)
        self.df["eat_calory"].fillna(self.df["eat_calory"].median(), inplace=True)
        self.df["Sleep_time"].fillna(self.df["Sleep_time"].median(), inplace=True)
        return self.df
    

import pandas as pd

class TimeSeriesLabeler:
    def __init__(self, term):
        self.term = term
        self.result_df = pd.DataFrame()
        self.labels = []

    def calculate_labels(self, df):
        for id in df['ID'].unique():
            df_id = df[df['ID'] == id].reset_index(drop=True)
            label = []

            for i in range(self.term, len(df_id)):
                previous_days = df_id['weight'].iloc[i - self.term:i]
                current_weight = df_id['weight'].iloc[i]
                if current_weight > previous_days.mean().round(3):
                    label.append(1)
                elif current_weight == previous_days.mean().round(3):
                    label.append(0)
                elif current_weight < previous_days.mean().round(3):
                    label.append(2)
                else:
                    print("error")

            self.labels.extend(label)

    def create_result_df(self, df):
        for id in df['ID'].unique():
            df_id = df[df['ID'] == id].reset_index(drop=True)
            df_id_drop = df_id.drop(index=range(self.term))
            self.result_df = pd.concat([self.result_df, df_id_drop])
        self.result_df['labels']= self.labels
        
    def get_labels(self):
        return self.labels

    def get_result_df(self):
        return self.result_df



In [6]:
processor = DataProcessor(df)
df_all=processor.result


In [7]:
df_all["height"].fillna(df_all["height"].mean(), inplace=True)
df_all["weight"].fillna(df_all["weight"].median(), inplace=True)

In [8]:
df_all.isnull().sum()

ID                  0
collect_datetime    0
gender              0
grade               0
height              0
weight              0
step_count          0
burned calory       0
eat_calory          0
Sleep_time          0
서포터여부               0
dtype: int64

In [9]:
df_all

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부
0,#AAGQKY,2022-07-14,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
1,#AAGQKY,2022-07-15,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
2,#AAGQKY,2022-07-16,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
3,#AAGQKY,2022-07-17,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
4,#AAGQKY,2022-07-18,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
...,...,...,...,...,...,...,...,...,...,...,...
57507,#ZZLSSL,2023-01-07,1,6,153.0,68.00,3579.0,53.55,701.0,480.0,1
57508,#ZZLSSL,2023-01-08,1,6,153.0,68.00,3579.0,53.55,701.0,480.0,1
57509,#ZZLSSL,2023-01-09,1,6,153.0,68.00,3579.0,53.55,701.0,480.0,1
57510,#ZZLSSL,2023-01-10,1,6,153.0,68.00,3579.0,53.55,701.0,480.0,1


In [10]:
df_all_new=df_all.dropna()

In [11]:
df_all_new.head()

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부
0,#AAGQKY,2022-07-14,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
1,#AAGQKY,2022-07-15,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
2,#AAGQKY,2022-07-16,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
3,#AAGQKY,2022-07-17,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
4,#AAGQKY,2022-07-18,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2


In [12]:
df_all_new[df_all_new.isna( ).any(axis=1)]

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부


In [13]:
df_all.head()

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부
0,#AAGQKY,2022-07-14,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
1,#AAGQKY,2022-07-15,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
2,#AAGQKY,2022-07-16,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
3,#AAGQKY,2022-07-17,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2
4,#AAGQKY,2022-07-18,2,4,148.0,45.15,3579.0,53.55,701.0,480.0,2


In [14]:
df_all_new['BMI']=df_all_new['weight']/(df_all_new['height']*0.01)**2

In [15]:
df_all_new.to_csv("../의미 있는 data/전처리 통합본.csv",encoding='cp949',index=False)

In [16]:
df_all=pd.read_csv("../의미 있는 data/전처리 통합본.csv",encoding="cp949")

# 남자, 여자 군집으로 나누기

In [17]:
df_boy=df_all[df_all['gender']==1]

In [18]:
df_girl=df_all[df_all['gender']==2]

In [19]:
df_boy.to_csv("../의미 있는 data/df_boy.csv",encoding='cp949',index=False)

In [20]:
df_girl.to_csv("../의미 있는 data/df_girl.csv",encoding='cp949',index=False)

# 학년별 군집 나누기

In [21]:
df_grade4=df_all[df_all['grade']==4]

In [22]:
df_grade5=df_all[df_all['grade']==5]

In [23]:
df_grade6=df_all[df_all['grade']==6]

In [24]:
df_grade4.to_csv("../의미 있는 data/df_grade4.csv",encoding='cp949',index=False)
df_grade5.to_csv("../의미 있는 data/df_grade5.csv",encoding='cp949',index=False)
df_grade6.to_csv("../의미 있는 data/df_grade6.csv",encoding='cp949',index=False)

# 저체중, 정상체중, 과체중

In [25]:
df_init=df_all[df_all['collect_datetime']=='2022-07-14']

In [26]:
df_init

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부,BMI
0,#AAGQKY,2022-07-14,2,4,148.000000,45.15,3579.0,53.55,701.0,480.0,2,20.612673
182,#ACGRQP,2022-07-14,2,4,136.016667,28.10,3579.0,53.55,701.0,480.0,2,15.188751
364,#AEGMOD,2022-07-14,2,4,132.000000,29.00,3579.0,53.55,701.0,480.0,1,16.643710
546,#AJXZPA,2022-07-14,1,5,136.100000,31.80,3579.0,53.55,701.0,480.0,2,17.167651
728,#AMMIRA,2022-07-14,2,5,147.500000,37.30,3579.0,53.55,701.0,480.0,1,17.144499
...,...,...,...,...,...,...,...,...,...,...,...,...
56602,#ZNPAMO,2022-07-14,2,5,137.666667,30.00,3579.0,53.55,701.0,480.0,1,15.829371
56784,#ZNRYPQ,2022-07-14,1,5,148.000000,36.00,3579.0,53.55,701.0,480.0,1,16.435354
56966,#ZRKRKP,2022-07-14,1,4,121.100000,24.00,3579.0,53.55,701.0,480.0,1,16.365262
57148,#ZWOHRX,2022-07-14,2,6,156.700000,47.90,3579.0,53.55,701.0,480.0,1,19.507316


In [27]:
df_thin_ID=df_init[df_init['BMI']<15]
df_origin_ID = df_init[(df_init['BMI'] >= 15) & (df_init['BMI'] <= 23)]
df_over_ID=df_init[df_init['BMI']>23]

In [28]:
target_thin_ids=df_thin_ID['ID'].tolist()
target_origin_ids=df_thin_ID['ID'].tolist()
target_over_ids=df_thin_ID['ID'].tolist()

In [29]:
df_all_thin = df_all[df_all['ID'].isin(target_thin_ids)]
df_all_origin = df_all[df_all['ID'].isin(target_origin_ids)]
df_all_over = df_all[df_all['ID'].isin(target_over_ids)]

In [30]:
df_all_thin

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,서포터여부,BMI
1820,#BBLXZS,2022-07-14,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
1821,#BBLXZS,2022-07-15,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
1822,#BBLXZS,2022-07-16,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
1823,#BBLXZS,2022-07-17,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
1824,#BBLXZS,2022-07-18,2,4,142.0,29.0,3579.0,53.55,701.0,480.0,2,14.382067
...,...,...,...,...,...,...,...,...,...,...,...,...
55869,#ZFPNFC,2023-01-07,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
55870,#ZFPNFC,2023-01-08,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
55871,#ZFPNFC,2023-01-09,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049
55872,#ZFPNFC,2023-01-10,2,5,151.0,33.0,3579.0,53.55,701.0,480.0,3,14.473049


In [31]:
"""df_thin=df_all[df_all['BMI']<15]
df_origin = df_all[(df_all['BMI'] >= 15) & (df_all['BMI'] <= 23)]
df_over=df_all[df_all['BMI']>23]"""

"df_thin=df_all[df_all['BMI']<15]\ndf_origin = df_all[(df_all['BMI'] >= 15) & (df_all['BMI'] <= 23)]\ndf_over=df_all[df_all['BMI']>23]"

In [32]:
df_all_thin.to_csv("../의미 있는 data/df_thin.csv",encoding='cp949',index=False)
df_all_origin.to_csv("../의미 있는 data/df_origin.csv",encoding='cp949',index=False)
df_all_over.to_csv("../의미 있는 data/df_over.csv",encoding='cp949',index=False)

# 리워드별 그룹나누기

In [33]:
reward_1=df_all[df_all['서포터여부']==1]
reward_2=df_all[df_all['서포터여부']==2]
reward_3=df_all[df_all['서포터여부']==3]

In [34]:
reward_1.to_csv("../의미 있는 data/reward_1.csv",encoding='cp949',index=False)
reward_2.to_csv("../의미 있는 data/reward_2.csv",encoding='cp949',index=False)
reward_3.to_csv("../의미 있는 data/reward_3.csv",encoding='cp949',index=False)